## Assignment 1 - Alpha algorithm

##### Input - list of tuples of str's: [(str1, ..., strL), (strJ,...strK), ..., (strM,...,strN)]
##### Output - set of tuples of tuples str's {((str1,...,strI), (strL,...,strK)),...}

In [ ]:
from petri_net import PetriNet
from subprocess import check_call

In [ ]:
log = [['a', 'b', 'c', 'd'], ['a', 'c', 'b', 'd'], ['a', 'e', 'd']]

#### Step 4. Make dataset

1. Define functions for causality, parallel and unrelated pairs<br>
2. Create footprint matrix<br>
3. Create pairs of events<br>
4. Iterate until no non-maximal pairs available

In [ ]:

def get_start_end_activities(log):
    start_activities = set()
    end_activities = set()
    for trace in log:
        if trace:
            start_activities.add(trace[0])
            end_activities.add(trace[-1])
    return start_activities, end_activities

def get_directly_follows_pairs(log):
    directly_follows = set()
    for trace in log:
        for i in range(len(trace) - 1):
            directly_follows.add((trace[i], trace[i + 1]))
    return directly_follows

def get_causal_relations(directly_follows):
    causal_relations = set()
    parallel_relations = set()
    for a, b in directly_follows:
        if (b, a) not in directly_follows:
            causal_relations.add((a, b))
        else:
            parallel_relations.add((a, b))
            parallel_relations.add((b, a))
    return causal_relations, parallel_relations

def get_alpha_relations(log):
    directly_follows = get_directly_follows_pairs(log)
    causal_relations, parallel_relations = get_causal_relations(directly_follows)

    alpha_relations = set()
    activities = {act for trace in log for act in trace}

    for act1 in activities:
        for act2 in activities:
            if act1 != act2:
                act1_to = {b for (a, b) in causal_relations if a == act1}
                to_act2 = {a for (a, b) in causal_relations if b == act2}
                if act1_to & to_act2:
                    alpha_relations.add(((act1,), tuple(act1_to & to_act2)))

    return alpha_relations

def alpha_miner(log):
    start_activities, end_activities = get_start_end_activities(log)
    alpha_relations = get_alpha_relations(log)

    relations = set()
    for start in start_activities:
        for follow_set in alpha_relations:
            if start in follow_set[0]:
                relations.add(((start,), follow_set[1]))

    final_relations = set()
    for (a, b) in relations:
        for end in end_activities:
            if end in b:
                final_relations.add((a, (end,)))
            else:
                final_relations.add((a, b))

    return final_relations

log = [['a', 'b', 'c', 'd'], ['a', 'c', 'b', 'd'], ['a', 'e', 'd']]

events = log
start_nodes, end_nodes = get_start_end_activities(log)
result = alpha_miner(log)

In [ ]:
result

{(('a',), ('c', 'e')),
 (('a',), ('e', 'b')),
 (('c', 'e'), ('d',)),
 (('e', 'b'), ('d',))}

Expected output (order irrelevant): {(('a',), ('c', 'e')),
 (('a',), ('e', 'b')),
 (('c', 'e'), ('d',)),
 (('e', 'b'), ('d',))}

#### Let's check the result with PetriNet drawer

In [ ]:
pn = PetriNet()
filename = 'my_first_alpha_miner'
pn.generate_with_alpha(tl=events,
                       ti=start_nodes,
                       to=end_nodes,
                       yl=result,
                       dotfile="{}.dot".format(filename))
check_call(["dot", "-Tpng", "{}.dot".format(filename),"-o", "{}.png".format(filename)])

TypeError: unsupported operand type(s) for -: 'list' and 'set'

## Another log for testing

Input:

In [ ]:
log = [('a', 'b', 'e', 'f'),
 ('a', 'b', 'e', 'c', 'd', 'b', 'f'),
 ('a', 'b', 'c', 'e', 'd', 'b', 'f'),
 ('a', 'b', 'c', 'd', 'e', 'b', 'f'),
 ('a', 'e', 'b', 'c', 'd', 'b', 'f')]
log

[('a', 'b', 'e', 'f'),
 ('a', 'b', 'e', 'c', 'd', 'b', 'f'),
 ('a', 'b', 'c', 'e', 'd', 'b', 'f'),
 ('a', 'b', 'c', 'd', 'e', 'b', 'f'),
 ('a', 'e', 'b', 'c', 'd', 'b', 'f')]

In [ ]:
result = alpha_miner(log)

In [ ]:
result

{(('a',), ('e',)),
 (('a', 'd'), ('b',)),
 (('b',), ('c', 'f')),
 (('c',), ('d',)),
 (('e',), ('f',))}

Expected output:

In [ ]:
dataset = {(('a',), ('e',)), (('a', 'd'), ('b',)), (('b',), ('c', 'f')), (('c',), ('d',)), (('e',), ('f',))}
dataset

{(('a',), ('e',)),
 (('a', 'd'), ('b',)),
 (('b',), ('c', 'f')),
 (('c',), ('d',)),
 (('e',), ('f',))}